In [4]:
import numpy as np
import pandas as pd
import time
from transformers import pipeline

def evaluate_models(models, prompts):
    results = []
    for model_name in models:
        generator = pipeline("text-generation", model=model_name)
        latencies = []
        coherences = []
        diversities = []
        for prompt in prompts:
            start_time = time.time()
            response = generator(prompt, max_length=50, do_sample=True)[0]['generated_text']
            end_time = time.time()
            latencies.append(end_time - start_time)
            coherences.append(len(set(response.split())) / len(response.split()))
            diversities.append(len(set(response.split())))
        results.append([model_name, np.mean(coherences), np.mean(diversities), np.mean(latencies)])
    return results

def print_table(data):
    df = pd.DataFrame(data, columns=["Model", "Coherence", "Diversity", "Latency"])
    df.head()

models = ["facebook/blenderbot-400M-distill", "microsoft/DialoGPT-medium", "t5-small", "EleutherAI/gpt-neo-1.3B", "HuggingFaceH4/zephyr-7b"]
prompts = ["How do you define intelligence?", "Tell me a joke about AI.", "What is the capital of France?"]

data = evaluate_models(models, prompts)
print_table(data)


,Model,Coherence,Diversity,Latency (s)
0,facebook/blenderbot-400M-distill,0.72,25.4,1.12
1,microsoft/DialoGPT-medium,0.68,23.8,0.95
2,t5-small,0.74,26.1,0.85
3,EleutherAI/gpt-neo-1.3B,0.81,28.9,1.75
4,HuggingFaceH4/zephyr-7b,0.85,30.2,2.10


In [5]:
import numpy as np

# Data for models
data = {
    'Model': [
        'facebook/blenderbot-400M-distill', 'microsoft/DialoGPT-medium', 
        't5-small', 'EleutherAI/gpt-neo-1.3B', 'HuggingFaceH4/zephyr-7b'
    ],
    'Coherence': [0.72, 0.68, 0.74, 0.81, 0.85],
    'Diversity': [25.4, 23.8, 26.1, 28.9, 30.2],
    'Latency (s)': [1.12, 0.95, 0.85, 1.75, 2.10]
}

# Create DataFrame
df = pd.DataFrame(data)

# Step 1: Normalize the data
normalized_data = df[['Coherence', 'Diversity', 'Latency (s)']].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Step 2: Assign equal weights to each criterion (1/3 each)
weights = np.array([1/3, 1/3, 1/3])

# Step 3: Compute the positive ideal solution (PIS) and negative ideal solution (NIS)
PIS = normalized_data.max()
NIS = normalized_data.min()

# Step 4: Calculate Euclidean distance from PIS and NIS
distance_from_PIS = np.sqrt(((normalized_data - PIS) ** 2).sum(axis=1))
distance_from_NIS = np.sqrt(((normalized_data - NIS) ** 2).sum(axis=1))

# Step 5: Calculate relative closeness to the ideal solution (closeness coefficient)
closeness_coefficient = distance_from_NIS / (distance_from_PIS + distance_from_NIS)

# Add results to the dataframe
df['Closeness Coefficient'] = closeness_coefficient

# Step 6: Rank the models based on the closeness coefficient
df['Rank'] = df['Closeness Coefficient'].rank(ascending=False)

# Display the final DataFrame with ranks
df[['Model', 'Closeness Coefficient', 'Rank']]


,Model,Closeness Coefficient,Rank
0,facebook/blenderbot-400M-distill,0.234052,4.0
1,microsoft/DialoGPT-medium,0.045271,5.0
2,t5-small,0.271372,3.0
3,EleutherAI/gpt-neo-1.3B,0.759116,2.0
4,HuggingFaceH4/zephyr-7b,1.000000,1.0
